# Clustering Crypto

In [16]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
from pathlib import Path
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [17]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [18]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
r = requests.get(url)
response = r.json()
data = pd.DataFrame(response["Data"]).transpose()
data.head()

,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,MaxSupply,MktCapPenalty,IsUsedInDefi,IsUsedInNft,PlatformType,AlgorithmType,Difficulty,BuiltOn,SmartContractAddress,DecimalPoints
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,42,0,0,0,blockchain,scrypt,0.915831,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,300,0,0,0,token,NaN,NaN,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,-1,0,0,0,blockchain,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,-1,0,0,0,blockchain,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# Alternatively, use the provided csv file:
# file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
csv_path = Path("../Starter_Files/Resources/crypto_data.csv")
crypto_df= pd.read_csv(csv_path,index_col=0)
crypto_df.head()


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [21]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
cryptos = (crypto_df[['CoinName','Algorithm','IsTrading', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply']])
cryptos.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [22]:
# Keep only cryptocurrencies that are trading
trading_cryptos = cryptos.loc[crypto_df['IsTrading'] == True]
trading_cryptos.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [23]:
# Keep only cryptocurrencies with a working algorithm
wa_cryptos= trading_cryptos.loc[~trading_cryptos['Algorithm'].str.contains('N/A')]
wa_cryptos.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [24]:
# Remove the "IsTrading" column
wa_cryptos.drop(columns=['IsTrading'], inplace=True)
wa_cryptos.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [25]:
# Remove rows with at least 1 null value
wa_cryptos = wa_cryptos.dropna()
wa_cryptos.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [26]:
# Remove rows with cryptocurrencies having no coins mined
no_coin_mined = wa_cryptos.loc[wa_cryptos['TotalCoinsMined'] > 0]
no_coin_mined.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [27]:
# Drop rows where there are 'N/A' text values
df = no_coin_mined[~no_coin_mined.Algorithm.str.contains('N/A', na=False)]
df = df[~no_coin_mined.ProofType.str.contains('N/A', na=False)]
df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [28]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
coin_names = df[["CoinName"]]
coin_names.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [29]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
df.drop(columns=['CoinName'], inplace=True)
df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [30]:
# Create dummy variables for text features
x = pd.get_dummies(df, columns=['Algorithm', 'ProofType'])
x.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
# Standardize data
x_scaled = StandardScaler().fit_transform(x)
print(x_scaled[0:1])

[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

### Reducing Dimensions Using PCA

In [32]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
pca

PCA(n_components=3)

In [33]:
# Create a DataFrame with the principal components data
df_pca = pca.fit_transform(x_scaled)
df_pca

array([[-0.33723839,  1.08309222, -0.55396539],
       [-0.32059236,  1.08304463, -0.55425405],
       [ 2.30711527,  1.5592933 , -0.63031871],
       ...,
       [ 0.32011629, -2.36272243,  0.31334516],
       [-0.13192154, -2.15221142,  0.42508186],
       [-0.29808404,  0.81432171, -0.27703624]])

In [34]:
data_pca_df = pd.DataFrame(
    data=df_pca, columns=["PC 1", "PC 2", "PC 3"],index=coin_names.index)
data_pca_df.head()

,PC 1,PC 2,PC 3
42,-0.337238,1.083092,-0.553965
404,-0.320592,1.083045,-0.554254
1337,2.307115,1.559293,-0.630319
BTC,-0.137865,-1.257606,0.243669
ETH,-0.148517,-2.027059,0.384114


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [35]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(data_pca_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")


:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [36]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
model.fit(data_pca_df)
# Predict clusters
predictions = model.predict(data_pca_df)
data_pca_df["Class"] = model.labels_

# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = pd.concat([df, data_pca_df, coin_names], axis=1, join="inner")
clustered_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.337238,1.083092,-0.553965,0,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.320592,1.083045,-0.554254,0,404Coin
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.307115,1.559293,-0.630319,0,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.137865,-1.257606,0.243669,3,Bitcoin
ETH,Ethash,PoW,1.076842e+08,0,-0.148517,-2.027059,0.384114,3,Ethereum
...,...,...,...,...,...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000,2.491418,1.018293,0.129804,0,ZEPHYR
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000,-0.335287,1.082942,-0.553998,0,Gapcoin
BDX,CryptoNight,PoW,9.802226e+08,1400222610,0.320116,-2.362722,0.313345,3,Beldex
ZEN,Equihash,PoW,7.296538e+06,21000000,-0.131922,-2.152211,0.425082,3,Horizen


### Visualizing Results

#### Scatter Plot with Tradable Cryptocurrencies

In [37]:
# Scale data to create the scatter plot
scaling_df = pd.DataFrame(clustered_df, columns=['TotalCoinSupply', 'TotalCoinsMined'])
crypto_scaled = StandardScaler().fit_transform(scaling_df)
crypto_scaled

array([[-0.1528703 , -0.11710817],
       [-0.145009  , -0.09396955],
       [ 4.48942416,  0.52494561],
       ...,
       [-0.13217937, -0.09561336],
       [-0.15255998, -0.11694817],
       [-0.15285552, -0.11710536]])

In [38]:
#Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
plot_df = pd.DataFrame(data=crypto_scaled,
                       columns=['TotalCoinSupply', 'TotalCoinsMined'],
                       index=clustered_df.index)
plot_df["CoinName" ] = clustered_df["CoinName" ]
plot_df["Class" ] = clustered_df["Class" ]
plot_df.hvplot.scatter(x="TotalCoinsMined",
                       y="TotalCoinSupply",
                       xlabel="Total Cryptocurrency Coins Mined",
                       ylabel="Total Cryptocurrency Coin Supply",
                       by = "Class",
                       hover_cols=["CoinName"])Mined" and y="TotalCoinSupply"


SyntaxError: invalid syntax (3483844163.py, line 12)

#### Table of Tradable Cryptocurrencies

In [39]:
# Table with tradable cryptos
clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"],
                          sortable=True,
                          selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [40]:
# Print the total number of tradable cryptocurrencies
clustered_df['CoinName'].count()

532